In [ ]:
## Esse script lê dados de um arquivo Excel e atualiza esses dados em uma tabela SQL Server

import pandas as pd
import pyodbc
from datetime import datetime
from tqdm import tqdm  
import warnings
warnings.filterwarnings('ignore')

excel_path = 'SA1SEG.xlsx'  # caminho do excel

# 2. Conexão com SQL Server 
server = '**********'
database = '********'
username = '********'
password = '*********'

conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# 3. Ler dados do Excel
try:
    df = pd.read_excel(excel_path)
    # Converter datas e tratar valores nulos
    df = df.where(pd.notnull(df), None)
    print(f"Total de registros no Excel: {len(df)}")
except Exception as e:
    print(f"Erro ao ler Excel: {str(e)}")
    exit()



In [ ]:
# 4. Atualizar banco de dados
try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        updated_rows = 0
        
        # Barra de progresso
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Atualizando registros"):
            
            # UPDATE
            sql = """
            UPDATE SA1010
            SET 
                A1_SATIV2 = ?,
                A1_SATIV3 = ?,
                A1_SATIV4 = ?
            WHERE A1_COD+A1_LOJA = ? 
            """
            def tratar_valor(valor):
                if pd.isna(valor):
                    return ''  
                return '000' + str(int(valor))

            params = (
                tratar_valor(row['A1_SATIV2']),
                tratar_valor(row['A1_SATIV3']),
                tratar_valor(row['A1_SATIV4']),
                row['CHAVE']
            )

            cursor.execute(sql, params)
            updated_rows += cursor.rowcount
            
            ## Commit a cada 100 registros para melhor performance
            if updated_rows % 100 == 0:
                conn.commit()
        
        conn.commit()  # Commit final
        print(f"\nTotal de registros atualizados: {updated_rows}")

except pyodbc.Error as e:
    print(f"Erro no banco de dados: {str(e)}")
except Exception as e:
    print(f"Erro inesperado: {str(e)}")
finally:
    print("Processo concluído.")